<a href="https://colab.research.google.com/github/CoolandHot/colab_tricks/blob/main/spark_cluster_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Tutorial on setting up Spark Cluster](https://medium.com/@jootorres_11979/how-to-install-and-set-up-an-apache-spark-cluster-on-hadoop-18-04-b4d70650ed42)

In [ ]:
MASTER_IP = '172.28.0.2'
Slave0_IP = '172.28.0.3'

In [13]:
#@title prerequisites
%%capture
Path_of_JAVA_installation = '/usr/lib/jvm/java-8-openjdk-amd64'
Spark_folder = "/usr/local/spark"

!pip install pyspark
# findspark will locate Spark on the system and import it as a regular library.
!pip install -q findspark

# ping tool
!apt-get install iputils-ping
# SSH
!apt-get install openssh-server openssh-client

# Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Scala
!apt-get install scala
# Apache Spark with Hadoop
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!mv spark-3.2.0-bin-hadoop3.2 {Spark_folder}
# Spark UI
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


# check hostname to vm0
!hostname vm0
with open('/etc/hosts', 'w') as file_object:
    host_text = '''127.0.0.1	localhost
 ::1	localhost ip6-localhost ip6-loopback
 fe00::0	ip6-localnet
 ff00::0	ip6-mcastprefix
 ff02::1	ip6-allnodes
 ff02::2	ip6-allrouters
 ''' + f'{MASTER_IP}	vm0\n {Slave0_IP}	vm1'
    file_object.write(host_text)

%cd /usr/local/spark/conf
# copy the template to a .sh file
!cp spark-env.sh.template spark-env.sh
with open('spark-env.sh', 'a') as file_object:
    file_object.write(f'\nexport SPARK_MASTER_HOST="{MASTER_IP}" export JAVA_HOME={Path_of_JAVA_installation}\n')
with open('slaves', 'a') as file_object:
    file_object.write('\nvm0\n')
    file_object.write('vm1\n')

%cd /content/
with open('/root/.bashrc', 'a') as file_object:
    file_object.write('\nexport PATH = $PATH:/usr/local/spark/bin\n')
# sourcing the ~/.bashrc file
!source ~/.bashrc



import os
os.environ["JAVA_HOME"] = Path_of_JAVA_installation
os.environ["SPARK_HOME"] = Spark_folder
import findspark
findspark.init()

In [2]:
!java -version
!scala -version
!ping vm1 -c 3
findspark.find()

openjdk version "11.0.13" 2021-10-19
OpenJDK Runtime Environment (build 11.0.13+8-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.13+8-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
Scala code runner version 2.11.12 -- Copyright 2002-2017, LAMP/EPFL
PING vm1 (172.28.0.3) 56(84) bytes of data.
64 bytes from vm1 (172.28.0.3): icmp_seq=1 ttl=64 time=0.079 ms
64 bytes from vm1 (172.28.0.3): icmp_seq=2 ttl=64 time=0.086 ms
64 bytes from vm1 (172.28.0.3): icmp_seq=3 ttl=64 time=0.062 ms

--- vm1 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2077ms
rtt min/avg/max/mdev = 0.062/0.075/0.086/0.014 ms


'/usr/local/lib/python3.7/dist-packages/pyspark'

In [3]:
!mkdir -p ~/.ssh/
!ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa
!cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys

Generating public/private rsa key pair.
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:WY/qHArQwF4c8T6qJxBW0JYCkuk4n+/0+8BHYoBN1w0 root@vm0
The key's randomart image is:
+---[RSA 2048]----+
|++o =...Eo       |
|+o X +  . .      |
|o B = .   .      |
|++ + o   o o     |
|.+o.. = S . .    |
|. o. + + .       |
| . .+ o +        |
|  .ooo * .       |
|  .+. +o+        |
+----[SHA256]-----+


In [ ]:
!ssh-copy-id root@vm1

In [35]:
# login the vm0
!ssh vm1

ssh: Could not resolve hostname vm0: Name or service not known


In [49]:
%cd /usr/local/spark
!sbin/start-master.sh

/usr/local/spark/conf


In [5]:
# check to if the services started
!jps

1298 Jps
